# Imports

In [3]:
import pandas as pd

In [53]:
import numpy as np
import ast

In [81]:
import matplotlib.pyplot as plt
import seaborn as sns

# Download Dataset


In [207]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("rounakbanik/the-movies-dataset")

print("Path to dataset files:", path)

100%|██████████| 228M/228M [00:01<00:00, 138MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/rounakbanik/the-movies-dataset/versions/7


In [209]:
!mv /root/.cache/kagglehub/datasets/rounakbanik /content/rounakbanik

In [245]:
movies_meta = pd.read_csv('/content/rounakbanik/the-movies-dataset/versions/7/movies_metadata.csv')
credits_rk = pd.read_csv('/content/rounakbanik/the-movies-dataset/versions/7/credits.csv')

<ipython-input-245-705baafcea08>:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies_meta = pd.read_csv('/content/rounakbanik/the-movies-dataset/versions/7/movies_metadata.csv')


# Cleaning Data

In [246]:
print(movies_meta.shape)
movies_meta.head(1)

(45466, 24)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0


In [261]:
print(credits_rk.shape)
credits_rk.head(1)

(45476, 3)


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862


In [257]:
ids_faltantes = credits_rk[~credits_rk['id'].isin(movies_meta['id'])]
missing_movies_id = credits_rk[~credits_rk['id'].isin(movies_meta['id'])]
#print(ids_faltantes)
#missing_movies_id.head()


In [259]:
movies_meta['id'] = movies_meta['id'].astype(str)
credits_rk['id'] = credits_rk['id'].astype(str)

In [265]:
movies_meta = movies_meta.merge(credits_rk, on="id")
movies_meta.head(1)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,cast,crew
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."


In [266]:
remove_cols = ['belongs_to_collection', 'budget', 'homepage', 'original_language', 'revenue', 'spoken_languages',
               'status', 'tagline', 'video', 'poster_path', 'production_companies', 'production_countries', 'crew']
movies_meta = movies_meta.drop(columns=remove_cols)

In [267]:
print(movies_meta.shape)
movies_meta.head(1)

(45538, 13)


,adult,genres,id,imdb_id,original_title,overview,popularity,release_date,runtime,title,vote_average,vote_count,cast
0,False,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0,Toy Story,7.7,5415.0,"[{'cast_id': 14, 'character': 'Woody (voice)',..."


In [268]:
movies_meta.isnull().sum()

,0
adult,0
genres,0
id,0
imdb_id,17
original_title,0
overview,954
popularity,3
release_date,87
runtime,260
title,3


In [269]:
movies_meta.dropna(inplace=True)

## Formating some collunms



In [270]:
def convert(obj):
    """
    Converting String dictionary into a list.
    """
    L = []
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [271]:
movies_meta['genres'] = movies_meta['genres'].apply(convert)

In [230]:
movies_meta.head(1)

,adult,genres,id,imdb_id,original_title,overview,popularity,release_date,runtime,title,vote_average,vote_count
0,False,"[Animation, Comedy, Family]",862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0,Toy Story,7.7,5415.0


In [272]:
movies_meta.iloc[0]['cast']

"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender': 2, 'id': 31, 'name': 'Tom Hanks', 'order': 0, 'profile_path': '/pQFoyx7rp09CJTAb932F2g8Nlho.jpg'}, {'cast_id': 15, 'character': 'Buzz Lightyear (voice)', 'credit_id': '52fe4284c3a36847f8024f99', 'gender': 2, 'id': 12898, 'name': 'Tim Allen', 'order': 1, 'profile_path': '/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg'}, {'cast_id': 16, 'character': 'Mr. Potato Head (voice)', 'credit_id': '52fe4284c3a36847f8024f9d', 'gender': 2, 'id': 7167, 'name': 'Don Rickles', 'order': 2, 'profile_path': '/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg'}, {'cast_id': 17, 'character': 'Slinky Dog (voice)', 'credit_id': '52fe4284c3a36847f8024fa1', 'gender': 2, 'id': 12899, 'name': 'Jim Varney', 'order': 3, 'profile_path': '/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg'}, {'cast_id': 18, 'character': 'Rex (voice)', 'credit_id': '52fe4284c3a36847f8024fa5', 'gender': 2, 'id': 12900, 'name': 'Wallace Shawn', 'order': 4, 'profile_path': '/oGE6JqPP2xH4t

In [ ]:
def fetch_5_characters(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 5:
          L.append(i['name'])
          counter+=1
        else:
            break
    return L

In [273]:
movies_meta['cast'] = movies_meta['cast'].apply(fetch_5_characters)
movies_meta.head(1)

,adult,genres,id,imdb_id,original_title,overview,popularity,release_date,runtime,title,vote_average,vote_count,cast
0,False,"[Animation, Comedy, Family]",862,tt0114709,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,81.0,Toy Story,7.7,5415.0,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."


# Recommendation Algorithm

In [274]:
unique_genres = {}

def ExtractGenres(genres_list):
    for genre in genres_list:
        if genre not in unique_genres:
            unique_genres[genre] = 1
        else:
            unique_genres[genre] += 1

movies_meta["genres"].apply(ExtractGenres)
unique_genres

{'Animation': 1924,
 'Comedy': 12818,
 'Family': 2736,
 'Adventure': 3477,
 'Fantasy': 2298,
 'Romance': 6681,
 'Drama': 20057,
 'Action': 6569,
 'Crime': 4274,
 'Thriller': 7597,
 'Horror': 4663,
 'History': 1377,
 'Science Fiction': 3032,
 'Mystery': 2458,
 'War': 1312,
 'Foreign': 1602,
 'Music': 1590,
 'Documentary': 3881,
 'Western': 1034,
 'TV Movie': 752}

In [275]:
C = movies_meta['vote_average'].mean()
m = movies_meta['vote_count'].quantile(0.90)
def weighted_rating(row, m=m, C=C):
    v = row['vote_count']
    R = row['vote_average']
    return (v / (v + m) * R) + (m / (v + m) * C)

movies_meta['weighted_rating'] = movies_meta.apply(weighted_rating, axis=1)

In [ ]:
mood_mapping = {
    'Feliz': ['Comedy', 'Family', 'Animation'],
    'Triste': ['Drama', 'Romance'],
    'Romântico': ['Romance', 'Drama', 'Music'],
    'Empolgante': ['Action', 'Adventure', 'Thriller'],
    'Assustador': ['Horror', 'Thriller', 'Mystery'],
    'Aventureiro': ['Adventure', 'Fantasy', 'Science Fiction'],
    'Curioso': ['Documentary', 'History', 'Mystery'],
    'Tenso': ['Crime', 'Thriller', 'Mystery'],
    'Reflexivo': ['Drama', 'History', 'War'],
    'Nostálgico': ['Music', 'Family', 'Romance'],
    'Relaxado': ['Comedy', 'Animation', 'Documentary'],
    'Épico': ['War', 'History', 'Adventure', 'Western'],
    'Intrigante': ['Foreign', 'Mystery', 'Documentary'],
    'Leve': ['Comedy', 'Family', 'TV Movie']
}

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def recomendar_filmes(mood, favorite_movie=None):
    mood_genres = mood_mapping[mood]
    filtered_movies = movies_meta[movies_meta['genres'].apply(lambda x: any(genre in x for genre in mood_genres))]

    if favorite_movie and favorite_movie in movies_meta['title'].values:
        tfidf = TfidfVectorizer(stop_words='english')
        tfidf_matrix = tfidf.fit_transform(movies_meta['overview'].fillna(''))
        movie_idx = movies_meta[movies_meta['title'] == favorite_movie].index[0]
        cosine_sim = cosine_similarity(tfidf_matrix[movie_idx], tfidf_matrix)
        sim_scores = list(enumerate(cosine_sim[0]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
        top_indices = [i[0] for i in sim_scores[1:6]]  # Top 5
        return filtered_movies[filtered_movies.index.isin(top_indices)]


    return filtered_movies.sort_values(by='weighted_rating', ascending=False).head(5)

In [277]:
recomendar_filmes("Épico")

,adult,genres,id,imdb_id,original_title,overview,popularity,release_date,runtime,title,vote_average,vote_count,cast,weighted_rating
522,False,"[Drama, History, War]",424,tt0108052,Schindler's List,The true story of how businessman Oskar Schind...,41.725123,1993-11-29,195.0,Schindler's List,8.3,4436.0,"[Liam Neeson, Ben Kingsley, Ralph Fiennes, Car...",8.204756
5489,False,"[Fantasy, Adventure, Animation, Family]",129,tt0245429,千と千尋の神隠し,A ten year old girl who wanders away from her ...,41.048867,2001-07-20,125.0,Spirited Away,8.3,3968.0,"[Rumi Hiiragi, Miyu Irino, Mari Natsuki, Bunta...",8.193971
1157,False,"[Adventure, Action, Science Fiction]",1891,tt0080684,The Empire Strikes Back,"The epic saga continues as Luke Skywalker, in ...",19.470959,1980-05-17,124.0,The Empire Strikes Back,8.2,5998.0,"[Mark Hamill, Harrison Ford, Carrie Fisher, Bi...",8.131573
15510,False,"[Action, Thriller, Science Fiction, Mystery, A...",27205,tt1375666,Inception,"Cobb, a skilled thief who commits corporate es...",29.108149,2010-07-14,148.0,Inception,8.1,14075.0,"[Leonardo DiCaprio, Joseph Gordon-Levitt, Elle...",8.071544
22922,False,"[Adventure, Drama, Science Fiction]",157336,tt0816692,Interstellar,Interstellar chronicles the adventures of a gr...,32.213481,2014-11-05,169.0,Interstellar,8.1,11187.0,"[Matthew McConaughey, Jessica Chastain, Anne H...",8.064304
